In [237]:
import pandas as pd
import numpy as np
import re

In [130]:
# ZHVI All Homes (SFR, Condo/Co-op) Time Series ($) (by ZIP code)
csv_home_values = "Resources/Zip_Zhvi_AllHomes.csv"

# Median Sale Price - Seasonally Adjusted ($) (by ZIP code)
csv_home_sales = "Resources/Sale_Prices_Zip.csv"

# Monthly Home Sales (Number, Raw) (by ZIP code)
csv_number_sales = "Resources/Sale_Counts_Zip.csv"

# Foreclosure Resales (%) (by ZIP code)
csv_foreclosure_resales = "Resources/SalesPrevForeclosed_Share_Zip.csv"

home_values = pd.read_csv(csv_home_values, encoding="ISO-8859-1")
home_sales = pd.read_csv(csv_home_sales, encoding="ISO-8859-1")
number_sales = pd.read_csv(csv_number_sales, encoding="ISO-8859-1")
foreclosure_resales = pd.read_csv(csv_foreclosure_resales, encoding="ISO-8859-1")

# Suffixes:
# ZVHI = Zillow Home Value Index ($)
# MSR = Median Sales Price - Seasonally Adjusted ($)
# NS = Number of Sales
# FR = Foreclosure Resales (%)




In [131]:
# Python Dictionary to translate US States to Two letter codes
# Dictionary obtained from rogerallen (https://gist.github.com/rogerallen/1583593)
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

# Rename StateName columns
home_values = home_values.rename(columns={"RegionName":"ZIP Code",
                                          "2008-01":"2008-01_ZHVI",
                                          "2008-02":"2008-02_ZHVI"})
home_sales = home_sales.rename(columns={"RegionName":"ZIP Code",
                                        "StateName":"State"})
number_sales = number_sales.rename(columns={"RegionName":"ZIP Code",
                                            "StateName":"State",
                                            "seasAdj":"seasAdj_NS"})
foreclosure_resales = foreclosure_resales.rename(columns={"RegionName":"ZIP Code",
                                                          "StateName":"State"})

# Translate states into two letter codes
try:
    for index, row in home_sales.iterrows():
        state = home_sales.loc[index, "State"]
        abbr = us_state_abbrev[state]
        home_sales.loc[index, "State"] = abbr

    
    for index, row in number_sales.iterrows():
        state = number_sales.loc[index, "State"]
        abbr = us_state_abbrev[state]
        number_sales.loc[index, "State"] = abbr

    for index, row in foreclosure_resales.iterrows():
        state = foreclosure_resales.loc[index, "State"]
        abbr = us_state_abbrev[state]
        foreclosure_resales.loc[index, "State"] = abbr
except:
    pass

home_values.head()

,RegionID,ZIP Code,City,State,Metro,CountyName,SizeRank,2008-01_ZHVI,2008-02_ZHVI,2008-03,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
0,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,1,NaN,NaN,NaN,...,4.846133e+05,4.844847e+05,4.856167e+05,4.865600e+05,4.858847e+05,4.849017e+05,4.823117e+05,4.800917e+05,4.787937e+05,4.785953e+05
1,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,2,NaN,NaN,NaN,...,1.192562e+06,1.184811e+06,1.182110e+06,1.185962e+06,1.188839e+06,1.181644e+06,1.163996e+06,1.148232e+06,1.148030e+06,1.163213e+06
2,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,3,269106.0,268326.0,267699.333333,...,3.352377e+05,3.350893e+05,3.339180e+05,3.331563e+05,3.325493e+05,3.327513e+05,3.328630e+05,3.339837e+05,3.346290e+05,3.352343e+05
3,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4,NaN,NaN,NaN,...,6.303370e+05,6.304280e+05,6.328107e+05,6.339393e+05,6.326857e+05,6.306867e+05,6.281890e+05,6.261287e+05,6.251030e+05,6.240497e+05
4,91940,77449,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,5,128064.0,127776.0,127537.666667,...,1.814760e+05,1.818737e+05,1.817140e+05,1.818840e+05,1.823700e+05,1.831330e+05,1.839323e+05,1.843723e+05,1.851663e+05,1.863170e+05


In [324]:
#Merge the home_values, home_sales, number_sales and foreclosure_resales
merge1 = pd.merge(home_values, home_sales, on=["ZIP Code","State","RegionID"], how="outer", suffixes=("_ZHVI", "_MSR"))
merge2 = pd.merge(number_sales, foreclosure_resales, on=["ZIP Code","State","RegionID"], how="outer", suffixes=("_NS", "_FR"))
main_data = pd.merge(merge1, merge2, on=["ZIP Code","State","RegionID"], how="outer")

In [133]:
main_data.head()

,RegionID,ZIP Code,City,State,Metro,CountyName,SizeRank_ZHVI,2008-01_ZHVI,2008-02_ZHVI,2008-03_ZHVI,...,2019-02_FR,2019-03_FR,2019-04_FR,2019-05_FR,2019-06_FR,2019-07_FR,2019-08_FR,2019-09_FR,2019-10_FR,2019-11_FR
0,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,1.0,NaN,NaN,NaN,...,0.0000,0.0092,0.0064,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,3.0,269106.0,268326.0,267699.333333,...,0.0058,0.0083,0.0072,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4.0,NaN,NaN,NaN,...,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,91940,77449,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,5.0,128064.0,127776.0,127537.666667,...,0.0000,0.0123,0.0123,0.0181,0.006,NaN,NaN,NaN,NaN,NaN


In [299]:
#Function to get mean of the columns from minimum month and year to maximum month and year with different suffixes
#Used regular expressions
def findavg(data,year,suffix):
    x = []
    for i in data.columns:
        y = (re.findall(f'^{year}-([0-9][0-9])_{suffix}',i))
        [x.append(int(i)) for i in y]
    min_col = data.columns.get_loc(f"{year}-{min(x):02d}_{suffix}")
    max_col = data.columns.get_loc(f"{year}-{max(x):02d}_{suffix}")
    data[f"{year}_Avg_{suffix}"] = data.iloc[:,min_col:max_col+1].mean(axis=1)
    return data[f"{year}_Avg_{suffix}"]

In [300]:
# Grabbed the 'RegionID', 'ZIP Code', 'City', 'State', 'CountyName' columns from main_data
new_main_data = main_data[['RegionID', 'ZIP Code', 'City', 'State', 'CountyName']]
new_main_data.head()

,RegionID,ZIP Code,City,State,CountyName
0,84654,60657,Chicago,IL,Cook County
1,61637,10023,New York,NY,New York County
2,91982,77494,Katy,TX,Harris County
3,84616,60614,Chicago,IL,Cook County
4,91940,77449,Katy,TX,Harris County


In [301]:
#Join all indexes across different years to new_main_data
years = [i for i in range(2008,2020,1)]
suffixes = ["ZHVI","MSR","NS","FR"]

for year in years:
    for suffix in suffixes:
        new_main_data = new_main_data.join(findavg(main_data,year,suffix))


In [315]:
new_main_data.head()
#new_main_data.to_csv("Output/main_data.csv", index=False, header=True)

,RegionID,ZIP Code,City,State,CountyName,2008_Avg_ZHVI,2008_Avg_MSR,2008_Avg_NS,2008_Avg_FR,2009_Avg_ZHVI,...,2017_Avg_NS,2017_Avg_FR,2018_Avg_ZHVI,2018_Avg_MSR,2018_Avg_NS,2018_Avg_FR,2019_Avg_ZHVI,2019_Avg_MSR,2019_Avg_NS,2019_Avg_FR
0,84654,60657,Chicago,IL,Cook County,479900.645833,376410.0,146.9,0.00000,466041.694444,...,213.000000,0.004208,4.886200e+05,3.704417e+05,179.750000,0.005367,4.835098e+05,4.097091e+05,131.090909,0.006700
1,61637,10023,New York,NY,New York County,NaN,NaN,NaN,NaN,833515.733333,...,75.333333,NaN,1.216848e+06,1.115458e+06,77.500000,NaN,1.176796e+06,1.146527e+06,71.909091,NaN
2,91982,77494,Katy,TX,Harris County,267059.388889,243020.0,79.2,0.03512,267583.416667,...,257.666667,0.002967,3.329190e+05,3.316750e+05,253.916667,0.001367,3.340930e+05,3.250500e+05,242.400000,0.007025
3,84616,60614,Chicago,IL,Cook County,606633.729167,NaN,NaN,NaN,594972.111111,...,215.400000,0.002430,6.361214e+05,4.334167e+05,197.416667,0.001183,6.297365e+05,5.015000e+05,153.000000,0.006750
4,91940,77449,Katy,TX,Harris County,125067.777778,NaN,125.7,0.24388,119876.111111,...,157.083333,0.012083,1.760530e+05,1.764667e+05,158.083333,0.003492,1.830033e+05,1.878727e+05,162.818182,0.011450


In [303]:
#Group the data by State and get mean acroos different zipcodes
state_data =new_main_data[
                     ['State', '2008_Avg_ZHVI','2009_Avg_ZHVI', '2010_Avg_ZHVI', '2011_Avg_ZHVI',
                      '2012_Avg_ZHVI', '2013_Avg_ZHVI', '2014_Avg_ZHVI', '2015_Avg_ZHVI', '2016_Avg_ZHVI',
                      '2017_Avg_ZHVI', '2018_Avg_ZHVI', '2019_Avg_ZHVI', '2008_Avg_MSR','2009_Avg_MSR', 
                      '2010_Avg_MSR', '2011_Avg_MSR', '2012_Avg_MSR','2013_Avg_MSR', '2014_Avg_MSR', 
                      '2015_Avg_MSR', '2016_Avg_MSR','2017_Avg_MSR', '2018_Avg_MSR', '2019_Avg_MSR', 
                      '2008_Avg_NS','2009_Avg_NS', '2010_Avg_NS', '2011_Avg_NS', '2012_Avg_NS','2013_Avg_NS', 
                      '2014_Avg_NS', '2015_Avg_NS', '2016_Avg_NS','2017_Avg_NS', '2018_Avg_NS', '2019_Avg_NS', 
                      '2008_Avg_FR', '2009_Avg_FR', '2010_Avg_FR', '2011_Avg_FR', '2012_Avg_FR','2013_Avg_FR', 
                      '2014_Avg_FR', '2015_Avg_FR', '2016_Avg_FR','2017_Avg_FR', '2018_Avg_FR', '2019_Avg_FR']
                    ].groupby("State").mean()
state_data.head()

,2008_Avg_ZHVI,2009_Avg_ZHVI,2010_Avg_ZHVI,2011_Avg_ZHVI,2012_Avg_ZHVI,2013_Avg_ZHVI,2014_Avg_ZHVI,2015_Avg_ZHVI,2016_Avg_ZHVI,2017_Avg_ZHVI,...,2010_Avg_FR,2011_Avg_FR,2012_Avg_FR,2013_Avg_FR,2014_Avg_FR,2015_Avg_FR,2016_Avg_FR,2017_Avg_FR,2018_Avg_FR,2019_Avg_FR
State,,,,,,,,,,,,,,,,,,,,,
AK,257757.731674,255277.256173,255265.845679,252652.662037,257154.490616,261483.093559,260587.859272,263911.006225,260905.361420,265698.318409,...,0.017953,0.015444,0.013250,0.015368,0.010808,0.014436,0.007723,0.008110,0.008051,0.015248
AL,142287.102771,137912.212703,132560.954521,127438.048756,124757.877755,124216.831304,123857.456570,126152.310004,126710.281543,129905.843310,...,0.028620,0.024541,0.022358,0.036341,0.038174,0.036396,0.029826,0.024794,0.021197,0.018580
AR,109428.506791,105079.607528,102444.345622,100620.504960,100541.646575,101893.995841,102221.952317,102801.309430,104403.715475,106864.871102,...,0.034028,0.024479,0.011072,0.017768,0.024253,0.012681,0.012304,0.011055,0.011784,0.012021
AZ,257791.880246,222270.168886,208525.831265,192496.057547,192090.256004,210370.328371,219667.363194,224545.767161,235508.485779,245929.407627,...,0.138657,0.132389,0.072265,0.059775,0.046469,0.031105,0.021606,0.016913,0.011268,0.009412
CA,468233.711450,423004.034608,422985.651603,401367.360069,406446.894985,446805.162324,476875.116913,503387.102992,536483.483562,569431.051039,...,0.212939,0.205428,0.145055,0.065362,0.036880,0.028936,0.022030,0.013541,0.009642,0.006600


In [307]:
#Get divorce-rate and convert to dataframe
divorce_csv = "Resources/divorce-rate.xlsx"

divorce_rate = pd.read_excel(csv, encoding="ISO-8859-1")
#Rates are based on provisional counts of divorces by state of occurrence.  
#Rates are per 1,000 total population residing in area. 
#Population enumerated as of April 1 for 1990, 2000, and 2010 and estimated as of July 1 for all other years
#Source: CDC/NCHS, National Vital Statistics System.

In [308]:
try:
    for index, row in divorce_rate.iterrows():
        state = divorce_rate.loc[index, "State"]
        abbr = us_state_abbrev[state]
        divorce_rate.loc[index, "State"] = abbr

except:
    pass
divorce_rate = divorce_rate.set_index("State")
years = [i for i in range(2008,2020,1)]
new_divorce = divorce_rate[years[0:-1]]
[new_divorce.rename(columns = {i:f"{i}_Div Rate"}, inplace = True) \
             for i in range(2008,2020,1)]
new_divorce = new_divorce.replace("---",np.nan)

In [309]:
merge = state_data.join(new_divorce, on="State")
# for index, row in new_merge.iterrows():
#         nc = new_merge.loc["NC", :]

In [310]:
merge[["2018_Avg_ZHVI", "2018_Avg_NS", "2018_Avg_ZHVI", "2018_Avg_MSR", "2018_Avg_FR", "2018_Div Rate"]].corr()

,2018_Avg_ZHVI,2018_Avg_NS,2018_Avg_ZHVI,2018_Avg_MSR,2018_Avg_FR,2018_Div Rate
2018_Avg_ZHVI,1.000000,0.456000,1.000000,0.972145,0.023705,-0.046551
2018_Avg_NS,0.456000,1.000000,0.456000,0.385747,0.096901,0.110624
2018_Avg_ZHVI,1.000000,0.456000,1.000000,0.972145,0.023705,-0.046551
2018_Avg_MSR,0.972145,0.385747,0.972145,1.000000,-0.061005,-0.072662
2018_Avg_FR,0.023705,0.096901,0.023705,-0.061005,1.000000,-0.203831
2018_Div Rate,-0.046551,0.110624,-0.046551,-0.072662,-0.203831,1.000000


In [323]:
unemployment_csv = "Resources/Cleaned_BLS_Unemployment-Rate-by-State_2008-2019_1-11-2020.csv"

unemployment_data = pd.read_csv(unemployment_csv, encoding="ISO-8859-1")
unemployment_data = unemployment_data.rename(columns={"State Abbreviation":"State"}).set_index("State")
unemployment_data.head()

,Data,Series Id,Adjusted?,Area,Area Type,State/Region/Division,Measure,Years,2008,2009,...,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116
State,,,,,,,,,,,,,,,,,,,,,
AL,Local Area Unemployment Statistics,LASST010000000000003,Seasonally Adjusted,Alabama,Statewide,Alabama,unemployment rate,2008 to 2019,5.72,10.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AK,Local Area Unemployment Statistics,LASST020000000000003,Seasonally Adjusted,Alaska,Statewide,Alaska,unemployment rate,2008 to 2019,6.71,7.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZ,Local Area Unemployment Statistics,LASST040000000000003,Seasonally Adjusted,Arizona,Statewide,Arizona,unemployment rate,2008 to 2019,6.14,9.93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AR,Local Area Unemployment Statistics,LASST050000000000003,Seasonally Adjusted,Arkansas,Statewide,Arkansas,unemployment rate,2008 to 2019,5.50,7.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CA,Local Area Unemployment Statistics,LASST060000000000003,Seasonally Adjusted,California,Statewide,California,unemployment rate,2008 to 2019,7.29,11.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [320]:
unemployment_data = unemployment_data[[str(i) for i in range(2008,2020,1)]]
for i in range(2008,2020,1):
    unemployment_data = unemployment_data.rename(columns={str(i):f"{i}_unemployment"})
unemployment_data.head()

,2008_unemployment,2009_unemployment,2010_unemployment,2011_unemployment,2012_unemployment,2013_unemployment,2014_unemployment,2015_unemployment,2016_unemployment,2017_unemployment,2018_unemployment,2019_unemployment
State,,,,,,,,,,,,
AL,5.72,10.99,10.54,9.62,7.98,7.22,6.76,6.08,5.85,4.43,3.94,3.37
AK,6.71,7.73,7.88,7.57,7.13,7.00,6.87,6.53,6.85,6.94,6.60,6.35
AZ,6.14,9.93,10.35,9.50,8.33,7.74,6.80,6.05,5.40,4.89,4.80,4.93
AR,5.50,7.83,8.19,8.29,7.58,7.22,6.02,5.03,3.95,3.71,3.70,3.57
CA,7.29,11.14,12.23,11.71,10.38,8.94,7.50,6.20,5.48,4.78,4.19,4.13


In [327]:
biggest_data = merge.join(unemployment_data, on="State").round(2)
biggest_data.head()

,2008_Avg_ZHVI,2009_Avg_ZHVI,2010_Avg_ZHVI,2011_Avg_ZHVI,2012_Avg_ZHVI,2013_Avg_ZHVI,2014_Avg_ZHVI,2015_Avg_ZHVI,2016_Avg_ZHVI,2017_Avg_ZHVI,...,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116
State,,,,,,,,,,,,,,,,,,,,,
AK,257757.73,255277.26,255265.85,252652.66,257154.49,261483.09,260587.86,263911.01,260905.36,265698.32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AL,142287.10,137912.21,132560.95,127438.05,124757.88,124216.83,123857.46,126152.31,126710.28,129905.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AR,109428.51,105079.61,102444.35,100620.50,100541.65,101894.00,102221.95,102801.31,104403.72,106864.87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZ,257791.88,222270.17,208525.83,192496.06,192090.26,210370.33,219667.36,224545.77,235508.49,245929.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CA,468233.71,423004.03,422985.65,401367.36,406446.89,446805.16,476875.12,503387.10,536483.48,569431.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [328]:
biggest_data.to_csv("Output/new_main_data.csv", index=True, header=True)

In [326]:
biggest_data.round(2)

,2008_Avg_ZHVI,2009_Avg_ZHVI,2010_Avg_ZHVI,2011_Avg_ZHVI,2012_Avg_ZHVI,2013_Avg_ZHVI,2014_Avg_ZHVI,2015_Avg_ZHVI,2016_Avg_ZHVI,2017_Avg_ZHVI,...,2010_unemployment,2011_unemployment,2012_unemployment,2013_unemployment,2014_unemployment,2015_unemployment,2016_unemployment,2017_unemployment,2018_unemployment,2019_unemployment
State,,,,,,,,,,,,,,,,,,,,,
AK,257757.73,255277.26,255265.85,252652.66,257154.49,261483.09,260587.86,263911.01,260905.36,265698.32,...,7.88,7.57,7.13,7.00,6.87,6.53,6.85,6.94,6.60,6.35
AL,142287.10,137912.21,132560.95,127438.05,124757.88,124216.83,123857.46,126152.31,126710.28,129905.84,...,10.54,9.62,7.98,7.22,6.76,6.08,5.85,4.43,3.94,3.37
AR,109428.51,105079.61,102444.35,100620.50,100541.65,101894.00,102221.95,102801.31,104403.72,106864.87,...,8.19,8.29,7.58,7.22,6.02,5.03,3.95,3.71,3.70,3.57
AZ,257791.88,222270.17,208525.83,192496.06,192090.26,210370.33,219667.36,224545.77,235508.49,245929.41,...,10.35,9.50,8.33,7.74,6.80,6.05,5.40,4.89,4.80,4.93
CA,468233.71,423004.03,422985.65,401367.36,406446.89,446805.16,476875.12,503387.10,536483.48,569431.05,...,12.23,11.71,10.38,8.94,7.50,6.20,5.48,4.78,4.19,4.13
CO,253618.98,245395.78,241199.47,237452.88,239758.26,255112.32,270234.28,292758.44,313756.49,331518.95,...,8.73,8.36,7.89,6.85,4.99,3.84,3.22,2.70,3.25,3.09
CT,347249.22,323905.39,313118.01,300227.80,288950.20,293717.92,294941.32,295011.95,298319.43,301334.04,...,9.13,8.83,8.35,7.81,6.63,5.73,5.16,4.70,4.12,3.72
DC,483599.19,458940.07,459613.42,459278.67,467900.16,506845.48,544902.23,563037.73,584612.03,609986.67,...,9.41,10.16,9.04,8.49,7.77,6.88,6.10,6.06,5.56,5.51
DE,300988.79,281541.65,272534.59,256029.80,248251.03,249140.89,258817.41,263802.46,270780.88,275315.41,...,8.42,7.50,7.21,6.69,5.72,4.85,4.52,4.48,3.78,3.41
